In [0]:
import numpy as np
import tensorflow as tf
import random

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 20
batch_size = 500

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [3]:
x = tf.placeholder(tf.float32, [None, 784], name='x')
# [batch, height, width, channels]
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, 10], name='y')
keep_prob = tf.placeholder(tf.float32, name='dropout')

with tf.name_scope('Layer1'):
  # shape = [height, width, in_channels, out_channels]
  W1 = tf.get_variable("W1", shape=[3, 3, 1, 32], 
                         initializer=tf.keras.initializers.he_normal())
  #    Conv     -> (?, 28, 28, 32)
  #    Pool     -> (?, 14, 14, 32)
  L1 = tf.nn.conv2d(x_img, W1, strides=[1, 1, 1, 1], padding='SAME')
  L1 = tf.nn.relu(L1)
  L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
  L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
    
  tf.summary.histogram("W1", W1)
  tf.summary.histogram("L1", L1)
    
with tf.name_scope("Layer2"):
  W2 = tf.get_variable("W2", shape=[3, 3, 32, 64], 
                         initializer=tf.contrib.layers.xavier_initializer())
  #    Conv     -> (?, 14, 14, 32)
  #    Pool     -> (?, 7, 7, 64)
  L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
  L2 = tf.nn.relu(L2)
  L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
  L2_flat = tf.reshape(L2, [-1, 7*7*64])
  L2_flat = tf.nn.dropout(L2_flat, keep_prob=keep_prob)
    
  tf.summary.histogram("W2", W2)
  tf.summary.histogram("L2", L2)
  tf.summary.histogram("L2_flat", L2_flat)
    
    
# Final FC 7x7x64 inputs -> 10 outputs
with tf.name_scope("Layer3"):
  W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10],
                   initializer=tf.contrib.layers.xavier_initializer())
  b = tf.Variable(tf.random_normal([10]))
  logits = tf.matmul(L2_flat, W3) + b
  logits = tf.nn.dropout(logits, keep_prob=keep_prob)

# define cost/loss & optimizer
with tf.name_scope("Cost"):
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
                  logits=logits, labels=y))

  tf.summary.scalar("cost", cost)

with tf.name_scope("Train"):
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
  merged_summary = tf.summary.merge_all()
  writer = tf.summary.FileWriter('./logs/Lab11/MNIST_99')
  writer.add_graph(sess.graph)
  
  sess.run(tf.global_variables_initializer())
  print("Learning started. It takes sometime.")
  for epoch in range(training_epochs):
    avg_cost = 0
    total_step = int(mnist.train.num_examples / batch_size)
    
    for step in range(total_step):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      feed_dict = {x: batch_xs, y:batch_ys, keep_prob:0.7}
      c, summary, _ = sess.run([cost, merged_summary, optimizer],
                                                feed_dict=feed_dict)
      
      writer.add_summary(summary, global_step=step)
      avg_cost += c/total_step
    
    print('Epoch: {:04d} cost = {:.9f}'.format(epoch+1, avg_cost))
  print("Learning Finished.")
  
  # Test
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  print('Accuracy:', sess.run(accuracy, feed_dict={
            x: mnist.test.images, y: mnist.test.labels, keep_prob:1.0}))

Learning started. It takes sometime.
Epoch: 0001 cost = 1.026158202
Epoch: 0002 cost = 0.577513492
Epoch: 0003 cost = 0.518939194
Epoch: 0004 cost = 0.491625099
Epoch: 0005 cost = 0.480847320
Epoch: 0006 cost = 0.465018183
Epoch: 0007 cost = 0.457581831
Epoch: 0008 cost = 0.450545028
Epoch: 0009 cost = 0.447184119
Epoch: 0010 cost = 0.443379170
Epoch: 0011 cost = 0.440651572
Epoch: 0012 cost = 0.437010284
Epoch: 0013 cost = 0.430650615
Epoch: 0014 cost = 0.434531700
Epoch: 0015 cost = 0.429622169
Epoch: 0016 cost = 0.430165812
Epoch: 0017 cost = 0.420060166
Epoch: 0018 cost = 0.420900879
Epoch: 0019 cost = 0.425347940
Epoch: 0020 cost = 0.422472340
Learning Finished.
Accuracy: 0.9911
